In [20]:
from pathlib import Path
import hashlib, shutil, subprocess, os
from datetime import datetime

REPO   = Path(r"C:\Users\Nic\Desktop\NHLPredictor")
DESK   = Path(r"C:\Users\Nic\Desktop")
NBNAME = "PredictNHL.ipynb"
EXTS   = {".csv", ".json", ".xlsx"}
BRANCH = "main"

# --- Force a CSV to "update" even if content is the same ---
FORCE_REFRESH_CSV = True
CSV_WHITELIST = []  # e.g. ["predictions.csv", "enhanced_game_data.csv"]; empty = all CSVs in repo

def sha(p):
    import hashlib
    h=hashlib.sha256()
    with p.open("rb") as f:
        for b in iter(lambda:f.read(1<<20), b""): h.update(b)
    return h.hexdigest()

def bump_csv_bytes(path: Path):
    """Toggle trailing newline to force a harmless byte change."""
    b = path.read_bytes()
    if b.endswith(b"\n"):
        path.write_bytes(b.rstrip(b"\n"))      # remove last newline
    else:
        path.write_bytes(b + b"\n")            # add last newline

def run(cmd):
    r = subprocess.run(cmd, cwd=str(REPO), capture_output=True, text=True)
    if r.stdout: print(r.stdout.strip())
    if r.stderr and r.returncode != 0: print(r.stderr.strip())
    return r.returncode

assert REPO.exists()
os.chdir(REPO)

# 1) Build targets: notebook + tracked csv/json/xlsx
targets = {NBNAME}
targets |= {p.name for p in REPO.iterdir() if p.is_file() and p.suffix.lower() in EXTS}

# 2) Copy Desktop -> repo when content differs
changed = []
for name in sorted(targets):
    src, dst = DESK/name, REPO/name
    if src.exists():
        if (not dst.exists()) or sha(src) != sha(dst):
            shutil.copy2(src, dst); changed.append(name)

# 3) Optionally force-refresh CSVs (toggle EOF newline)
if FORCE_REFRESH_CSV:
    csvs = [p for p in REPO.iterdir() if p.is_file() and p.suffix.lower()==".csv"]
    if CSV_WHITELIST:
        csvs = [p for p in csvs if p.name in CSV_WHITELIST]
    for p in csvs:
        # only bump if not already modified by step 2 (to avoid double-noise)
        if p.name not in changed:
            bump_csv_bytes(p); changed.append(p.name)

# 4) Commit & push (only if something actually changed)
if changed:
    run(["git","add"] + changed)
    msg = f"Auto update (nb+data) - {datetime.now():%Y-%m-%d %H:%M:%S}"
    run(["git","commit","-m", msg])
    run(["git","pull","--rebase","origin", BRANCH])
    run(["git","push","-u","origin", BRANCH])
    print("Pushed:", ", ".join(changed))
else:
    print("Nothing to update. (No content changes and force-refresh off or nothing matched.)")



[main 7a0f5bd] Auto update (nb+data) - 2025-12-03 07:55:05
 3 files changed, 2819 insertions(+), 2598 deletions(-)
Current branch main is up to date.
branch 'main' set up to track 'origin/main'.
Pushed: PredictNHL.ipynb, enhanced_game_data.csv, predictions.csv


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels
!pip install lightgbm

   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   --- ------------------------------------ 1.3/15.6 MB 8.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/15.6 MB 8.4 MB/s eta 0:00:02
   ------------ --------------------------- 5.0/15.6 MB 8.6 MB/s eta 0:00:02
   ---------------- ----------------------- 6.6/15.6 MB 8.4 MB/s eta 0:00:02
   ---------------------- ----------------- 8.9/15.6 MB 9.1 MB/s eta 0:00:01
   ---------------------------- ----------- 11.0/15.6 MB 9.2 MB/s eta 0:00:01
   --------------------------------- ------ 13.1/15.6 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  15.5/15.6 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 15.6/15.6 MB 9.4 MB/s  0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# ========================
# Standard library
# ========================
import os
import time
import json
import random
import warnings
import re
import unicodedata
from datetime import datetime, timedelta
from io import StringIO

# ========================
# Data & numerical
# ========================
import numpy as np
import pandas as pd

# ========================
# Web scraping
# ========================
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.common.by import By

# ========================
# Visualization
# ========================
import matplotlib.pyplot as plt
import seaborn as sns

# ========================
# Statistics
# ========================
import scipy.stats as stats
from scipy.stats import loguniform, uniform
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ========================
# Scikit-learn
# ========================
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    TimeSeriesSplit,
    StratifiedKFold
)

from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    OneHotEncoder,
    LabelEncoder
)

from sklearn.impute import SimpleImputer

from sklearn.feature_selection import (
    mutual_info_classif,
    RFECV,
    SelectFromModel,
    SelectKBest
)

from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    brier_score_loss,
    accuracy_score,
    log_loss,
    classification_report,
    make_scorer
)

from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# ========================
# Boosting models
# ========================
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# ========================
# Imbalanced learning
# ========================
from imblearn.over_sampling import SMOTE

# ========================
# Model persistence
# ========================
import joblib

In [2]:
# EXTRACT BOTH SEASONS
SEASONS = [2025, 2026]

FEATURES_STATS = {
    'points_pct': 'PTS%',
    'srs': 'SRS',
    'goals_for_per_game': 'GF/G',
    'goals_against_per_game': 'GA/G',
    'power_play_pct': 'PP%',
    'pen_kill_pct': 'PK%',
    'save_pct': 'SV%',
    'shots': 'S',
    'shots_against': 'SA'
}

FEATURES_5V5 = {
    'corsi_pct_5on5': 'CF%',
    'fenwick_pct_5on5': 'FF%',
    'exp_on_goals_for': 'xGF',
    'exp_on_goals_against': 'xGA',
    'hdsc_for_pct': 'HDCF%',
    'pdo': 'PDO'
}

_clean = lambda n: n.replace('*', '').strip() if n else ''
_to_num = lambda x: pd.to_numeric(str(x).replace('%', '').replace(',', '').strip(), errors='coerce')

def _find_table_in_comments(soup, table_id: str):
    # hockey-reference hides some advanced tables inside HTML comments
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if table_id in c:
            return BeautifulSoup(c, 'html.parser').find('table', id=table_id)
    return None

def _get_stat_cell(row, data_stat: str):
    # keep your fallback logic (lower/upper) for safety
    return (
        row.find("td", {"data-stat": data_stat})
        or row.find("td", {"data-stat": data_stat.lower()})
        or row.find("td", {"data-stat": data_stat.upper()})
    )

def scrape_season(season):
    url = f"https://www.hockey-reference.com/leagues/NHL_{season}.html"

    with webdriver.Chrome() as driver:
        driver.get(url)
        time.sleep(2.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

    # ---------- TEAM STATS ----------
    table_stats = soup.find('table', id='stats')
    records = []

    for i, row in enumerate(table_stats.find_all('tr')[1:]):
        cols = row.find_all('td')
        if not cols:
            continue

        team = _clean(cols[0].text)
        if team == "League Average":
            continue

        rec = {'Team': team, 'Rank': i + 1}
        for k, lbl in FEATURES_STATS.items():
            el = _get_stat_cell(row, k)
            rec[lbl] = _to_num(el.text) if el else None

        records.append(rec)

    df_stats = pd.DataFrame(records)
    df_stats["Season"] = season

    # ---------- 5v5 ADV (IN COMMENTS) ----------
    table_adv = _find_table_in_comments(soup, 'stats_adv')
    adv_records = []

    # Safety: if hockey-reference changes layout, avoid hard crash
    if table_adv is not None:
        for row in table_adv.find_all('tr')[1:]:
            cols = row.find_all('td')
            if not cols:
                continue

            team = _clean(cols[0].text)
            if team == "League Average":
                continue

            rec = {'Team': team}
            for k, lbl in FEATURES_5V5.items():
                el = _get_stat_cell(row, k)
                rec[lbl] = _to_num(el.text) if el else None

            adv_records.append(rec)

    df_5v5 = pd.DataFrame(adv_records)
    df_5v5["Season"] = season

    # xGF% only if columns exist (prevents KeyError)
    if {"xGF", "xGA"}.issubset(df_5v5.columns) and not df_5v5.empty:
        df_5v5["xGF%"] = (df_5v5["xGF"] / (df_5v5["xGF"] + df_5v5["xGA"]) * 100).round(1)
    else:
        df_5v5["xGF%"] = np.nan

    # ---------- MERGE ----------
    df = (
        pd.merge(df_stats, df_5v5, on=["Team", "Season"], how="left")
          .sort_values("Rank")
          .reset_index(drop=True)
    )

    return df

# Keep your variables (important for downstream code)
df_2025 = scrape_season(2025)
df_2026 = scrape_season(2026)

# Combine both datasets (same output as before)
df_combined = (
    pd.concat([df_2025, df_2026], ignore_index=True)
      .sort_values(["Team", "Season"])
      .reset_index(drop=True)
)

In [3]:
UA = {"User-Agent": "Mozilla/5.0"}
TIMEZONE = "America/Toronto"

def to_csv_url(url: str) -> str:
    return re.sub(r"/results/(nhl-\d{4})$", r"/download/\1-UTC.csv", url)

def parse_datetime_column(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce", utc=True, dayfirst=True)
    miss = dt.isna()
    if miss.any():
        dt.loc[miss] = pd.to_datetime(s[miss], errors="coerce", utc=True, dayfirst=False)
    return dt

# ==========================
# TEAM NORMALIZATION SECTION
# ==========================
TEAM_NAME_MAP = {
    "Montreal Canadiens": "Montreal Canadiens",
    "Montréal Canadiens": "Montreal Canadiens",

    "Utah Hockey Club": "Utah Mammoth",
    "Utah HC": "Utah Mammoth",
    "Utah Hockey C": "Utah Mammoth",
    "Utah Mammoth": "Utah Mammoth",
}

def normalize_team_name(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip()
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    s = re.sub(r"\s{2,}", " ", s)
    return TEAM_NAME_MAP.get(s, s)

def load_game_results(url: str) -> pd.DataFrame:
    csv_url = to_csv_url(url)

    # One reliable path: download text then read it
    r = requests.get(csv_url, headers=UA, timeout=20)
    r.raise_for_status()
    df = pd.read_csv(StringIO(r.text))

    df.columns = [c.strip() for c in df.columns]
    lc = {c.lower(): c for c in df.columns}

    def _col(*names, default=None):
        for n in names:
            if n in lc:
                return df[lc[n]]
        return default

    out = pd.DataFrame({
        "Date_str": _col("date"),
        "Home Team": _col("home team", "home"),
        "Away Team": _col("away team", "away"),
    })

    def _extract_scores(series: pd.Series) -> pd.DataFrame:
        return series.astype(str).str.extract(r"(\d+)\s*[-–−]\s*(\d+)")

    # Scores (prefer explicit columns if present)
    if {"Home Score", "Away Score"}.issubset(df.columns):
        out["Home Score"] = pd.to_numeric(df["Home Score"], errors="coerce")
        out["Away Score"] = pd.to_numeric(df["Away Score"], errors="coerce")
    elif "Score" in df.columns:
        s = _extract_scores(df["Score"])
        out["Home Score"] = pd.to_numeric(s[0], errors="coerce")
        out["Away Score"] = pd.to_numeric(s[1], errors="coerce")
    else:
        out["Home Score"] = pd.NA
        out["Away Score"] = pd.NA

    # Fill missing from "result" if available
    if "result" in lc:
        miss = out["Home Score"].isna() | out["Away Score"].isna()
        s = _extract_scores(df[lc["result"]])
        out.loc[miss, "Home Score"] = pd.to_numeric(s[0], errors="coerce")
        out.loc[miss, "Away Score"] = pd.to_numeric(s[1], errors="coerce")

    out["Date_UTC"] = parse_datetime_column(out["Date_str"])
    out["Date_Local"] = out["Date_UTC"].dt.tz_convert(TIMEZONE)
    out["LocalDate"] = out["Date_Local"].dt.date

    # Clean rows
    out = (
        out.dropna(subset=["Date_UTC"])
           .sort_values("Date_UTC")
           .drop_duplicates(subset=["Date_UTC", "Home Team", "Away Team"], keep="last")
           .reset_index(drop=True)
    )

    # Normalize TEAM NAMES exactly ONCE
    out[["Home Team", "Away Team"]] = out[["Home Team", "Away Team"]].applymap(normalize_team_name)

    return out

In [4]:
# ================================================
# ==== BUILD df_all AFTER loading game results ====
# ================================================

def _load_season(url: str, season_label: str, weight: float) -> pd.DataFrame:
    df = load_game_results(url)
    df["Season"] = season_label
    df["weight"] = weight
    return df

df_2024 = _load_season("https://fixturedownload.com/results/nhl-2024", "2024-2025", 1.0)
df_2025 = _load_season("https://fixturedownload.com/results/nhl-2025", "2025-2026", 2.0)

df_all = pd.concat([df_2024, df_2025], ignore_index=True)

# Played flag
df_all["Played"] = df_all[["Home Score", "Away Score"]].notna().all(axis=1)

# Build master list
df_games_master = (
    df_all[["LocalDate", "Home Team", "Away Team", "Home Score", "Away Score", "Season", "Played"]]
      .rename(columns={"LocalDate": "Date"})
      .sort_values("Date")
      .reset_index(drop=True)
)

print("✔ df_all built:", df_all.shape)
print("✔ df_games_master built:", df_games_master.shape)

C:\Users\Nic\AppData\Local\Temp\ipykernel_25912\2439047771.py:93: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  out[["Home Team", "Away Team"]] = out[["Home Team", "Away Team"]].applymap(normalize_team_name)


✔ df_all built: (2624, 11)
✔ df_games_master built: (2624, 7)


C:\Users\Nic\AppData\Local\Temp\ipykernel_25912\2439047771.py:93: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  out[["Home Team", "Away Team"]] = out[["Home Team", "Away Team"]].applymap(normalize_team_name)


In [5]:
# =====================================
# ==== BUILD df_final (Played only) ===
# =====================================

df_final = (
    df_all.loc[df_all["Played"]]
          .copy()
          .assign(Date=lambda d: pd.to_datetime(d["LocalDate"], errors="coerce", utc=True).dt.tz_convert(None))
          .dropna(subset=["Date"])
          .sort_values("Date")
          .reset_index(drop=True)
)

print("✔ df_final built:", df_final.shape)
print(df_final.tail())

✔ df_final built: (2220, 12)
              Date_str             Home Team            Away Team Home Score  \
2215  06/02/2026 00:30   Tampa Bay Lightning     Florida Panthers        6.0   
2216  06/02/2026 00:00   Philadelphia Flyers      Ottawa Senators        1.0   
2217  06/02/2026 00:00   Washington Capitals  Nashville Predators        4.0   
2218  06/02/2026 00:00        Buffalo Sabres  Pittsburgh Penguins        2.0   
2219  06/02/2026 03:00  Vegas Golden Knights    Los Angeles Kings        4.0   

     Away Score                  Date_UTC                Date_Local  \
2215        1.0 2026-02-06 00:30:00+00:00 2026-02-05 19:30:00-05:00   
2216        2.0 2026-02-06 00:00:00+00:00 2026-02-05 19:00:00-05:00   
2217        2.0 2026-02-06 00:00:00+00:00 2026-02-05 19:00:00-05:00   
2218        5.0 2026-02-06 00:00:00+00:00 2026-02-05 19:00:00-05:00   
2219        1.0 2026-02-06 03:00:00+00:00 2026-02-05 22:00:00-05:00   

       LocalDate     Season  weight  Played       Date  
2215  

In [6]:
# Add Home Win flag if missing
if "Home Win" not in df_final.columns:
    df_final["Home Win"] = (df_final["Home Score"] > df_final["Away Score"]).astype(int)

df_final = df_final.sort_values("Date").reset_index(drop=True)

OFFSEASON_GAP_DAYS = 45

# Pre-allocate outputs (keep same column names)
home_last10, away_last10 = [], []
home_py, away_py = [], []
home_wr, away_wr = [], []
home_streak, away_streak = [], []
home_rest, away_rest = [], []

# State per (team, season)
wins_last10 = {}      # key -> list of last wins (0/1)
last_date = {}        # key -> last game date (normalized)
wins_games = {}       # key -> (wins, games)
streak = {}           # key -> current win streak

for _, row in df_final.iterrows():
    season = row["Season"]
    date = pd.Timestamp(row["Date"]).normalize()

    home = row["Home Team"]
    away = row["Away Team"]
    home_key = (home, season)
    away_key = (away, season)

    home_win = int(row["Home Win"])
    away_win = 1 - home_win

    # --- 1) LAST 10 WINS (before game) ---
    home_last10.append(sum(wins_last10.get(home_key, [])[-10:]))
    away_last10.append(sum(wins_last10.get(away_key, [])[-10:]))

    # --- 2) PLAYED YESTERDAY ---
    yday = date - pd.Timedelta(days=1)
    home_py.append(last_date.get(home_key) == yday)
    away_py.append(last_date.get(away_key) == yday)

    # --- 3) WIN RATE (before game) ---
    hw, hg = wins_games.get(home_key, (0, 0))
    aw, ag = wins_games.get(away_key, (0, 0))
    home_wr.append(hw / hg if hg else 0)
    away_wr.append(aw / ag if ag else 0)

    # --- 4) OVERALL WIN STREAK (before game) ---
    home_streak.append(streak.get(home_key, 0))
    away_streak.append(streak.get(away_key, 0))

    # --- 5) REST DAYS ---
    ph = last_date.get(home_key)
    pa = last_date.get(away_key)
    dh = (date - ph).days if ph is not None else None
    da = (date - pa).days if pa is not None else None
    home_rest.append(dh if dh is not None and dh <= OFFSEASON_GAP_DAYS else np.nan)
    away_rest.append(da if da is not None and da <= OFFSEASON_GAP_DAYS else np.nan)

    # ===== Update state AFTER computing "before game" features =====
    wins_last10.setdefault(home_key, []).append(home_win)
    wins_last10.setdefault(away_key, []).append(away_win)

    last_date[home_key] = date
    last_date[away_key] = date

    wins_games[home_key] = (hw + home_win, hg + 1)
    wins_games[away_key] = (aw + away_win, ag + 1)

    if home_win:
        streak[home_key] = streak.get(home_key, 0) + 1
        streak[away_key] = 0
    else:
        streak[home_key] = 0
        streak[away_key] = streak.get(away_key, 0) + 1

# Assign columns (same names as your original code)
df_final["Home Last 10 Wins"] = home_last10
df_final["Away Last 10 Wins"] = away_last10

df_final["Home Played Yesterday"] = home_py
df_final["Away Played Yesterday"] = away_py

df_final["Home Win Rate"] = home_wr
df_final["Away Win Rate"] = away_wr

df_final["Home Team Overall Win Streak Before Game"] = home_streak
df_final["Away Team Overall Win Streak Before Game"] = away_streak

df_final["Home Rest Days Since Last Game"] = home_rest
df_final["Away Rest Days Since Last Game"] = away_rest

In [7]:
# --- Diffs (OK) ---
df_final["Home Advantage"] = df_final["Home Win Rate"] - df_final["Away Win Rate"]
df_final["Win Streak Impact"] = (
    df_final["Home Team Overall Win Streak Before Game"]
    - df_final["Away Team Overall Win Streak Before Game"]
)
df_final["Last 10 Wins"] = df_final["Home Last 10 Wins"] - df_final["Away Last 10 Wins"]


# ============================================================
# MERGE CLEAN (Season_Key + Team_Key)  ✅
# ============================================================

def _team_key(s: str) -> str:
    s = "" if s is None else str(s)
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    return " ".join(s.strip().lower().split())

def _season_key(x):
    # "2024-2025" -> 2025 ; "2025-2026" -> 2026 ; 2025 -> 2025
    if x is None:
        return None
    xs = str(x)
    if "-" in xs:
        try:
            return int(xs.split("-")[1])
        except Exception:
            return None
    try:
        return int(float(xs))
    except Exception:
        return None

# --- 1) Keys côté df_final ---
df_final["Season_Key"] = df_final["Season"].map(_season_key)
df_final["Home_Team_Key"] = df_final["Home Team"].map(_team_key)
df_final["Away_Team_Key"] = df_final["Away Team"].map(_team_key)
df_final = df_final.dropna(subset=["Season_Key", "Home_Team_Key", "Away_Team_Key"])

# --- 2) Keys côté df_combined ---
stats = df_combined.copy()
stats["Season_Key"] = stats["Season"].map(_season_key)
stats["Team_Key"] = stats["Team"].map(_team_key)

# garder colonnes utiles + enlever doublons
stats = (
    stats.drop(columns=[c for c in ["Season", "Team"] if c in stats.columns])
         .drop_duplicates(subset=["Season_Key", "Team_Key"])
)

def _merge_side(df, side: str, key_col: str) -> pd.DataFrame:
    side_stats = (
        stats.add_prefix(f"{side}_")
             .rename(columns={f"{side}_Season_Key": "Season_Key", f"{side}_Team_Key": key_col})
    )
    return df.merge(side_stats, on=["Season_Key", key_col], how="left", validate="m:1")

# --- 3-4) Merge Home + Away ---
df_final = _merge_side(df_final, "Home", "Home_Team_Key")
df_final = _merge_side(df_final, "Away", "Away_Team_Key")

# --- 5) Quick check ---
cols_check = ["Home_PP%", "Away_PP%", "Home_xGF%", "Away_xGF%"]
for col in cols_check:
    if col in df_final.columns:
        print(col, "NA rate:", df_final[col].isna().mean().round(3))

Home_PP% NA rate: 0.018
Away_PP% NA rate: 0.018
Home_xGF% NA rate: 0.018
Away_xGF% NA rate: 0.018


In [8]:
SNAPSHOT_PATH = "team_snapshots.csv"
SNAPSHOT_SEASON = 2026

today = pd.Timestamp.now().normalize()

snapshot = (
    df_combined.loc[df_combined["Season"] == SNAPSHOT_SEASON]
        .copy()
        .assign(
            **{
                "Snapshot Date": today,
                "Team": lambda d: d["Team"].astype(str).str.strip()
            }
        )
)

if os.path.exists(SNAPSHOT_PATH):
    df_existing = pd.read_csv(SNAPSHOT_PATH, parse_dates=["Snapshot Date"])
    df_existing = df_existing[df_existing["Snapshot Date"] != today]
    snapshot = pd.concat([df_existing, snapshot], ignore_index=True)

snapshot.to_csv(SNAPSHOT_PATH, index=False)

print(f"✅ Snapshot saved for {today.date()}. Total entries: {len(snapshot)}")

✅ Snapshot saved for 2026-02-25. Total entries: 320


In [9]:
# ============================================================
# FEATURE BUILD (clean + shorter, no leakage)
# ============================================================

SNAPSHOT_CSV = "team_snapshots.csv"
USE_DATE_COL = "Date"
ROLL_WINDOWS = [5, 10, 20]
MIN_PERIODS = 3
SNAP_METRICS = ["PP%", "PK%", "SV%", "xGF%"]

# -----------------------
# 0) PREP / SAFETY
# -----------------------
df_final = df_final.loc[:, ~df_final.columns.duplicated()].copy()

if USE_DATE_COL not in df_final.columns:
    raise ValueError(f"Missing '{USE_DATE_COL}' in df_final.")

df_final[USE_DATE_COL] = pd.to_datetime(df_final[USE_DATE_COL], errors="coerce")
df_final = df_final.dropna(subset=[USE_DATE_COL]).sort_values(USE_DATE_COL).reset_index(drop=True)

for c in ["Home Team", "Away Team", "Season"]:
    if c not in df_final.columns:
        raise ValueError(f"Missing '{c}' in df_final.")

def _to_float_series(s):
    if s.dtype == "O":
        s = s.astype(str).str.replace("%", "", regex=False).str.replace(",", "", regex=False)
    return pd.to_numeric(s, errors="coerce")

def _team_key(s: str) -> str:
    s = "" if s is None else str(s)
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    return " ".join(s.strip().lower().split())

def _season_key(x):
    if x is None:
        return None
    xs = str(x)
    if "-" in xs:
        try:
            return int(xs.split("-")[1])
        except Exception:
            return None
    try:
        return int(float(xs))
    except Exception:
        return None

def prev_season_label(season_str: str) -> str:
    a, b = season_str.split("-")
    a, b = int(a), int(b)
    return f"{a-1}-{b-1}"

# -----------------------
# 1) LOAD SNAPSHOTS
# -----------------------
snapshots = pd.read_csv(SNAPSHOT_CSV, parse_dates=["Snapshot Date"])
snapshots["Team"] = snapshots["Team"].astype(str).str.strip()
snapshots["Season"] = snapshots["Season"].astype(str).str.strip()

for m in SNAP_METRICS:
    if m in snapshots.columns:
        snapshots[m] = _to_float_series(snapshots[m])

# -----------------------
# 2) BUILD PRIOR FINAL STATS (prev season final snapshot)
# -----------------------
if {"Season", "Team"}.issubset(df_combined.columns) is False:
    raise ValueError("df_combined must contain ['Season','Team', ...]")

final_2025_stats = df_combined.loc[df_combined["Season"] == 2025].copy()
final_2025_stats["Team"] = final_2025_stats["Team"].astype(str).str.strip()
final_2025_stats["Snapshot Date"] = pd.Timestamp("2025-07-01")
final_2025_stats["Season"] = "2024-2025"

keep_cols = ["Team", "Season", "Snapshot Date"] + [
    c for c in final_2025_stats.columns
    if c in SNAP_METRICS or c in ["Rank","PTS%","SRS","GF/G","GA/G","S","SA","CF%","FF%","xGF","xGA","HDCF%","PDO","xGF%"]
]
keep_cols = list(dict.fromkeys(keep_cols))
final_2025_stats = final_2025_stats.loc[:, keep_cols]

for m in SNAP_METRICS:
    if m in final_2025_stats.columns:
        final_2025_stats[m] = _to_float_series(final_2025_stats[m])

# -----------------------
# 3) MERGE SNAPSHOTS (NO-LEAK) via merge_asof
# -----------------------
def merge_snapshot_stats(df_games, df_snaps, prior_final_stats, team_col, prefix):
    df_games = df_games.copy()
    df_games["Season"] = df_games["Season"].astype(str).str.strip()
    df_games[team_col] = df_games[team_col].astype(str).str.strip()

    # Build keys for stable joins
    df_games[f"{prefix}_Team_Key"] = df_games[team_col].map(_team_key)
    df_snaps = df_snaps.copy()
    df_snaps["Team_Key"] = df_snaps["Team"].map(_team_key)

    # Same-season snapshots (<= game date)
    left = df_games[[USE_DATE_COL, "Season", f"{prefix}_Team_Key"]].rename(columns={f"{prefix}_Team_Key": "Team_Key"})
    left = left.sort_values(USE_DATE_COL)

    right = df_snaps.rename(columns={"Snapshot Date": "SnapDate"}).sort_values("SnapDate")

    merged = pd.merge_asof(
        left,
        right,
        left_on=USE_DATE_COL,
        right_on="SnapDate",
        by=["Season", "Team_Key"],
        direction="backward",
        allow_exact_matches=True
    )

    # Fallback: previous season final only for the specific known case (your original rule)
    fallback = prior_final_stats.copy()
    fallback["Team_Key"] = fallback["Team"].map(_team_key)

    # Determine which rows need fallback
    need = merged["SnapDate"].isna() & merged["Season"].str.contains("-", na=False)

    if need.any():
        prev = merged.loc[need, "Season"].map(prev_season_label)
        # only apply fallback when prev season == "2024-2025" (your exact behavior)
        use_fb = prev.eq("2024-2025")
        idx = merged.loc[need].index[use_fb]

        if len(idx):
            fb_pick = merged.loc[idx, ["Team_Key"]].merge(
                fallback, on="Team_Key", how="left"
            )
            # overwrite missing snapshot fields with fallback fields
            for c in fallback.columns:
                if c in ["Team_Key"]:
                    continue
                merged.loc[idx, c] = fb_pick[c].values
            merged.loc[idx, "SnapDate"] = fb_pick["Snapshot Date"].values

    # Add fallback flag: 1 if still missing OR if filled by fallback
    used_fallback = merged["SnapDate"].isna().astype(int)
    # (If you want to count “filled by fallback” as 1 too:)
    # used_fallback |= need.astype(int)  # optional, but your original flagged only when used

    # Prefix columns (except keys)
    keep = [c for c in merged.columns if c not in ["Season", "Team_Key", USE_DATE_COL]]
    pref = merged[keep].rename(columns=lambda c: f"{prefix}_{c}")
    out = pd.concat([df_games, pref], axis=1)
    out[f"{prefix}_Snapshot_Fallback"] = used_fallback.values
    return out

df_final = merge_snapshot_stats(df_final, snapshots, final_2025_stats, "Home Team", "Home")
df_final = merge_snapshot_stats(df_final, snapshots, final_2025_stats, "Away Team", "Away")
df_final = df_final.loc[:, ~df_final.columns.duplicated()].copy()

# Ensure required snapshot metric columns exist
missing_cols = [f"{side}_{m}" for side in ["Home", "Away"] for m in SNAP_METRICS if f"{side}_{m}" not in df_final.columns]
if missing_cols:
    raise ValueError(f"Missing required snapshot columns after merge: {missing_cols}")

# -----------------------
# 4) ROLLING RECENT FORM (SHIFTED) + DELTAS
# -----------------------
for m in SNAP_METRICS:
    df_final[f"Home_{m}"] = _to_float_series(df_final[f"Home_{m}"])
    df_final[f"Away_{m}"] = _to_float_series(df_final[f"Away_{m}"])

df_final = df_final.reset_index(drop=True)
df_final["Game_ID"] = df_final.index

def _side_long(side: str, team_col: str):
    cols = ["Game_ID", USE_DATE_COL, "Season", team_col] + [f"{side}_{m}" for m in SNAP_METRICS]
    out = df_final[cols].copy().rename(columns={team_col: "Team"})
    out["Side"] = side
    for m in SNAP_METRICS:
        out = out.rename(columns={f"{side}_{m}": m})
    return out

team_long = pd.concat(
    [_side_long("Home", "Home Team"), _side_long("Away", "Away Team")],
    ignore_index=True
).sort_values(["Team", USE_DATE_COL, "Game_ID"]).reset_index(drop=True)

for m in SNAP_METRICS:
    shifted = team_long.groupby("Team")[m].shift(1)
    for w in ROLL_WINDOWS:
        team_long[f"{m}_L{w}"] = shifted.groupby(team_long["Team"]).transform(
            lambda s: s.rolling(w, min_periods=MIN_PERIODS).mean()
        )

roll_cols = [f"{m}_L{w}" for m in SNAP_METRICS for w in ROLL_WINDOWS]

cols_to_drop = (
    [f"Home_{c}" for c in roll_cols] + [f"Away_{c}" for c in roll_cols] +
    [f"Delta_{m}_L{w}" for m in SNAP_METRICS for w in ROLL_WINDOWS]
)
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns], errors="ignore")

home_roll = team_long.loc[team_long["Side"] == "Home", ["Game_ID"] + roll_cols].rename(columns={c: f"Home_{c}" for c in roll_cols})
away_roll = team_long.loc[team_long["Side"] == "Away", ["Game_ID"] + roll_cols].rename(columns={c: f"Away_{c}" for c in roll_cols})

df_final = df_final.merge(home_roll, on="Game_ID", how="left").merge(away_roll, on="Game_ID", how="left")

for m in SNAP_METRICS:
    for w in ROLL_WINDOWS:
        df_final[f"Delta_{m}_L{w}"] = df_final[f"Home_{m}_L{w}"] - df_final[f"Away_{m}_L{w}"]

# Drop raw snapshot metrics (optional)
drop_snapshot_cols = [f"Home_{m}" for m in SNAP_METRICS] + [f"Away_{m}" for m in SNAP_METRICS]
df_final = df_final.drop(columns=[c for c in drop_snapshot_cols if c in df_final.columns], errors="ignore")

print("✅ Rolling + deltas created.")

# -----------------------
# 5) EXTRA FEATURES (Rank/SRS diffs + Opponent Strength)
# -----------------------
if {"Home_Rank", "Away_Rank"}.issubset(df_final.columns):
    df_final["Rank Difference"] = _to_float_series(df_final["Home_Rank"]) - _to_float_series(df_final["Away_Rank"])

if {"Home_SRS", "Away_SRS"}.issubset(df_final.columns):
    df_final["SRS_Diff"] = _to_float_series(df_final["Home_SRS"]) - _to_float_series(df_final["Away_SRS"])

def calculate_avg_opponent_rank(df):
    hist = {}
    home_strengths, away_strengths = [], []
    for _, row in df.iterrows():
        h, a = str(row["Home Team"]), str(row["Away Team"])
        hist.setdefault(h, [])
        hist.setdefault(a, [])

        home_strengths.append(np.mean(hist[h][-10:]) if hist[h] else np.nan)
        away_strengths.append(np.mean(hist[a][-10:]) if hist[a] else np.nan)

        ar = row.get("Away_Rank", np.nan)
        hr = row.get("Home_Rank", np.nan)
        if not pd.isna(ar):
            hist[h].append(float(ar))
        if not pd.isna(hr):
            hist[a].append(float(hr))
    return home_strengths, away_strengths

df_final = df_final.sort_values(USE_DATE_COL).reset_index(drop=True)
df_final["Home Opponent Strength"], df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(df_final)

print("✅ Feature build complete.")
print("Snapshot fallback rates:",
      "Home=", round(df_final["Home_Snapshot_Fallback"].mean(), 3),
      "Away=", round(df_final["Away_Snapshot_Fallback"].mean(), 3))

✅ Rolling + deltas created.
✅ Feature build complete.
Snapshot fallback rates: Home= 0.603 Away= 0.605


In [10]:
# ---------------- CONFIG ----------------
DATE_COL = "Date"
TARGET_COL = "Home Win"
TEST_HORIZON_DAYS = 14
MIN_TRAIN_GAMES = 500
CALIBRATION_BINS = 10

WINDOW_MODE = "L5"        # "L5" | "L10" | "L20" | "L5_L20" | "ALL"
CAL_MODE = "sigmoid"      # "none" | "sigmoid" | "isotonic"
CAL_CV = 3

# ---------------- BUILD BACKTEST DF ----------------
df_bt = (
    df_final.loc[:, ~df_final.columns.duplicated()]
            .copy()
)

df_bt[DATE_COL] = pd.to_datetime(df_bt[DATE_COL], errors="coerce")
df_bt = df_bt.dropna(subset=[DATE_COL, TARGET_COL]).sort_values(DATE_COL).reset_index(drop=True)

# keep played only (scores present)
if {"Home Score", "Away Score"}.issubset(df_bt.columns):
    df_bt = df_bt.dropna(subset=["Home Score", "Away Score"]).reset_index(drop=True)

# Build diffs
if {"Home Rest Days Since Last Game", "Away Rest Days Since Last Game"}.issubset(df_bt.columns):
    df_bt["Rest_Diff"] = df_bt["Home Rest Days Since Last Game"] - df_bt["Away Rest Days Since Last Game"]

if {"Home Played Yesterday", "Away Played Yesterday"}.issubset(df_bt.columns):
    df_bt["B2B_Diff"] = df_bt["Home Played Yesterday"].astype(int) - df_bt["Away Played Yesterday"].astype(int)

# ---- Rolling delta selection ----
all_delta = sorted([c for c in df_bt.columns if c.startswith("Delta_") and "_L" in c])

_mode_map = {
    "L5":      ["_L5"],
    "L10":     ["_L10"],
    "L20":     ["_L20"],
    "L5_L20":  ["_L5", "_L20"],
    "ALL":     None
}
if WINDOW_MODE not in _mode_map:
    raise ValueError("Bad WINDOW_MODE")

need = _mode_map[WINDOW_MODE]
FEAT_ROLLING_DELTAS = all_delta if need is None else [c for c in all_delta if any(t in c for t in need)]

# Other features
FEAT_DIFFS = [c for c in ["SRS_Diff", "Rank Difference", "Win Streak Impact", "Last 10 Wins", "Home Advantage"] if c in df_bt.columns]
FEAT_OPP   = [c for c in ["Home Opponent Strength", "Away Opponent Strength"] if c in df_bt.columns]
FEAT_REST  = [c for c in ["Rest_Diff", "B2B_Diff"] if c in df_bt.columns]

feature_cols = list(dict.fromkeys(FEAT_ROLLING_DELTAS + FEAT_DIFFS + FEAT_OPP + FEAT_REST))
if not feature_cols:
    raise ValueError("No features selected. Ensure Delta_*_L* exists (and WINDOW_MODE matches).")

print("\n=== Walk-forward Feature Set ===")
print("WINDOW_MODE:", WINDOW_MODE)
print("CAL_MODE:   ", CAL_MODE)
print(f"Rolling deltas: {len(FEAT_ROLLING_DELTAS)}")
print(f"Diffs:          {len(FEAT_DIFFS)} -> {FEAT_DIFFS}")
print(f"Opp strength:   {len(FEAT_OPP)} -> {FEAT_OPP}")
print(f"Rest/B2B:       {len(FEAT_REST)} -> {FEAT_REST}")
print(f"TOTAL:          {len(feature_cols)}")

# X/y
y_all = df_bt[TARGET_COL].astype(int).to_numpy()
X_all = df_bt[feature_cols].replace([np.inf, -np.inf], np.nan)

# ============================================================
# DROP GLOBAL ALL-NaN FEATURES (CRITICAL INTEGRITY FIX)
# ============================================================
global_all_nan = X_all.columns[X_all.isna().all()].tolist()
if global_all_nan:
    print("Dropping globally all-NaN features:", global_all_nan)
    X_all = X_all.drop(columns=global_all_nan)
    feature_cols = [c for c in feature_cols if c not in global_all_nan]

# ---------------- MODEL PIPELINE ----------------
base_clf = LogisticRegression(max_iter=4000, solver="lbfgs", C=1.0, random_state=42)

if CAL_MODE == "none":
    clf = base_clf
elif CAL_MODE in ("sigmoid", "isotonic"):
    clf = CalibratedClassifierCV(base_clf, method=CAL_MODE, cv=CAL_CV)
else:
    raise ValueError("Bad CAL_MODE")

pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", clf),
])

# ---------------- WALK-FORWARD ----------------
pred_rows = []
dates = df_bt[DATE_COL]

# unique start days (sorted)
unique_days = sorted(dates.dt.normalize().unique())

for ws in unique_days:
    we = ws + pd.Timedelta(days=TEST_HORIZON_DAYS)

    train_mask = dates < ws
    test_mask  = (dates >= ws) & (dates < we)

    if test_mask.sum() == 0 or train_mask.sum() < MIN_TRAIN_GAMES:
        continue

    X_tr, y_tr = X_all.loc[train_mask], y_all[train_mask]
    X_te, y_te = X_all.loc[test_mask],  y_all[test_mask]

    if X_tr.shape[1] == 0:
        continue

    pipe.fit(X_tr, y_tr)
    p = np.clip(pipe.predict_proba(X_te)[:, 1], 1e-6, 1 - 1e-6)

    block = df_bt.loc[test_mask, [DATE_COL, "Home Team", "Away Team", "Season"]].copy()
    block["y_true"] = y_te
    block["p_home"] = p
    pred_rows.append(block)

pred_df = pd.concat(pred_rows, ignore_index=True) if pred_rows else pd.DataFrame()

# ---------------- METRICS ----------------
overall_brier = brier_score_loss(pred_df["y_true"], pred_df["p_home"])
overall_roc   = roc_auc_score(pred_df["y_true"], pred_df["p_home"])
overall_ll    = log_loss(pred_df["y_true"], pred_df["p_home"])

print("\n=== WALK-FORWARD (overall) ===")
print(f"Brier:   {overall_brier:.5f}")
print(f"ROC-AUC: {overall_roc:.5f}")
print(f"LogLoss: {overall_ll:.5f}")

# Brier by month
pred_df["Month"] = pred_df[DATE_COL].dt.to_period("M").astype(str)
brier_by_month = (
    pred_df.groupby("Month")
           .apply(lambda g: brier_score_loss(g["y_true"], g["p_home"]))
           .reset_index(name="Brier")
)

# Calibration bins
pred_df["bin"] = pd.cut(pred_df["p_home"], bins=np.linspace(0, 1, CALIBRATION_BINS + 1), include_lowest=True)
cal_table = (
    pred_df.groupby("bin", observed=True)
           .agg(n=("y_true", "size"), p_mean=("p_home", "mean"), win_rate=("y_true", "mean"))
           .reset_index()
)

print("\n=== Brier by Month ===")
print(brier_by_month.to_string(index=False))

print("\n=== Calibration bins ===")
print(cal_table.to_string(index=False))

# Save outputs with tags so you can compare runs quickly
tag = f"{WINDOW_MODE}_{CAL_MODE}"
pred_df.to_csv(f"walkforward_predictions_{tag}.csv", index=False)
brier_by_month.to_csv(f"walkforward_brier_by_month_{tag}.csv", index=False)
cal_table.to_csv(f"walkforward_calibration_bins_{tag}.csv", index=False)

print(f"\n✅ Saved walk-forward outputs with tag: {tag}")


=== Walk-forward Feature Set ===
WINDOW_MODE: L5
CAL_MODE:    sigmoid
Rolling deltas: 4
Diffs:          5 -> ['SRS_Diff', 'Rank Difference', 'Win Streak Impact', 'Last 10 Wins', 'Home Advantage']
Opp strength:   2 -> ['Home Opponent Strength', 'Away Opponent Strength']
Rest/B2B:       2 -> ['Rest_Diff', 'B2B_Diff']
TOTAL:          13

=== WALK-FORWARD (overall) ===
Brier:   0.23609
ROC-AUC: 0.62641
LogLoss: 0.66486

=== Brier by Month ===
  Month    Brier
2024-12 0.204426
2025-01 0.246612
2025-02 0.240998
2025-03 0.219697
2025-04 0.231825
2025-10 0.247093
2025-11 0.240715
2025-12 0.248881
2026-01 0.233843
2026-02 0.188748

=== Calibration bins ===
       bin    n   p_mean  win_rate
(0.1, 0.2]   15 0.195402  0.000000
(0.2, 0.3]  227 0.264727  0.387665
(0.3, 0.4] 2032 0.360657  0.397146
(0.4, 0.5] 4738 0.454367  0.440692
(0.5, 0.6] 6681 0.550519  0.550666
(0.6, 0.7] 5505 0.643392  0.642507
(0.7, 0.8] 1888 0.736333  0.724047
(0.8, 0.9]  182 0.828320  0.763736

✅ Saved walk-forward output

C:\Users\Nic\AppData\Local\Temp\ipykernel_25912\3485064123.py:141: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: brier_score_loss(g["y_true"], g["p_home"]))


In [11]:
print(
    "Snapshot fallback rates:",
    *(f"{side}={df_final[f'{side}_Snapshot_Fallback'].mean():.3f}"
      for side in ["Home", "Away"])
)

Snapshot fallback rates: Home=0.603 Away=0.605


In [12]:
# ============================================================
# FINAL TRAINING — Logistic L2 vs ElasticNet (Brier-first)
# Requires: feature_cols from Walk-forward cell
# ============================================================

SEED = 42
CV_SPLITS = 6
GAP = 3
TEST_RATIO = 0.2
TARGET_COL = "Home Win"
USE_TEAM_DUMMIES = True

if "feature_cols" not in globals() or not feature_cols:
    raise RuntimeError("feature_cols missing. Run the Walk-forward cell first.")

random.seed(SEED)
np.random.seed(SEED)

df = df_final.sort_values("Date").reset_index(drop=True)

# Rebuild Rest/B2B diffs here (they exist in df_bt, not always in df_final)
if {"Home Rest Days Since Last Game", "Away Rest Days Since Last Game"}.issubset(df.columns):
    df["Rest_Diff"] = df["Home Rest Days Since Last Game"] - df["Away Rest Days Since Last Game"]
if {"Home Played Yesterday", "Away Played Yesterday"}.issubset(df.columns):
    df["B2B_Diff"] = df["Home Played Yesterday"].astype(int) - df["Away Played Yesterday"].astype(int)

need_cols = [TARGET_COL, "Home Score", "Away Score", "Home Team", "Away Team", "Date"]
df_base = df.dropna(subset=[c for c in need_cols if c in df.columns]).sort_values("Date").reset_index(drop=True)

# Keep a clean history object for prediction later (played only)
df_hist_for_pred = df_base.copy()

# Optional categorical team features
cat_features = []
if USE_TEAM_DUMMIES:
    df_base["Home_Team"] = df_base["Home Team"].astype("category")
    df_base["Away_Team"] = df_base["Away Team"].astype("category")
    cat_features = ["Home_Team", "Away_Team"]

# Rename numeric feature cols (spaces -> underscore)
rename_map = {c: c.replace(" ", "_") for c in feature_cols}
inv_rename_map = {v: k for k, v in rename_map.items()}
df_base = df_base.rename(columns=rename_map)
feat = [rename_map[c] for c in feature_cols]

X = df_base[feat + cat_features]
y = df_base[TARGET_COL].astype(int)

# Drop global all-NaN numeric features (keep meta aligned)
all_nan_global = X[feat].columns[X[feat].isna().all()].tolist()
if all_nan_global:
    print("Dropping all-NaN features:", all_nan_global)
    feat = [c for c in feat if c not in all_nan_global]
    X = df_base[feat + cat_features]
    dropped_original = [inv_rename_map[c] for c in all_nan_global if c in inv_rename_map]
    feature_cols = [c for c in feature_cols if c not in dropped_original]

cut = int((1 - TEST_RATIO) * len(df_base))
X_tr, X_te = X.iloc[:cut], X.iloc[cut:]
y_tr, y_te = y.iloc[:cut], y.iloc[cut:]

tscv = TimeSeriesSplit(n_splits=CV_SPLITS, gap=GAP)

# Preprocess: impute->scale for numeric; one-hot for teams
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]), feat),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ],
    remainder="drop"
)

def eval_model(model, Xt, yt):
    p = model.predict_proba(Xt)[:, 1]
    yhat = (p >= 0.5).astype(int)
    return {
        "F1": f1_score(yt, yhat, average="weighted"),
        "ROC_AUC": roc_auc_score(yt, p),
        "Brier": brier_score_loss(yt, p),
    }

def calibrate_and_pick(model, Xtr, ytr, Xte, yte, cv=5):
    base_m = eval_model(model, Xte, yte)
    cands = [("base", model, base_m)]
    for method in ["sigmoid", "isotonic"]:
        cal = CalibratedClassifierCV(model, method=method, cv=cv).fit(Xtr, ytr)
        cands.append((method, cal, eval_model(cal, Xte, yte)))
    return min(cands, key=lambda x: x[2]["Brier"]), cands

def run_search(pipe, params, n_iter):
    return RandomizedSearchCV(
        pipe,
        params,
        n_iter=n_iter,
        scoring="neg_brier_score",
        refit=True,
        cv=tscv,
        n_jobs=-1,
        verbose=1,
        random_state=SEED
    ).fit(X_tr, y_tr)

pipe_l2 = Pipeline([
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=4000, solver="lbfgs", random_state=SEED)),
])

pipe_en = Pipeline([
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=4000, solver="saga", penalty="elasticnet", random_state=SEED)),
])

PARAMS_L2 = {"clf__C": loguniform(1e-3, 1e1), "clf__class_weight": [None, "balanced"]}
PARAMS_EN = {"clf__C": loguniform(1e-3, 1e1), "clf__l1_ratio": uniform(0, 1), "clf__class_weight": [None, "balanced"]}

rs_l2 = run_search(pipe_l2, PARAMS_L2, n_iter=50)
rs_en = run_search(pipe_en, PARAMS_EN, n_iter=60)

best_l2, best_en = rs_l2.best_estimator_, rs_en.best_estimator_

m_l2 = eval_model(best_l2, X_te, y_te)
m_en = eval_model(best_en, X_te, y_te)

chosen = ("Logistic L2", best_l2, m_l2) if m_l2["Brier"] <= m_en["Brier"] else ("ElasticNet", best_en, m_en)
print("\nPre-calibration best:", chosen[0], chosen[2])

(best_cal_name, best_cal_model, best_cal_metrics), cal_cands = calibrate_and_pick(chosen[1], X_tr, y_tr, X_te, y_te, cv=5)

print("\nCalibration comparison:")
for name, mdl, met in cal_cands:
    print(f"{name:8s}  Brier={met['Brier']:.5f}  ROC={met['ROC_AUC']:.5f}  F1={met['F1']:.5f}")

print(f"\n✅ FINAL: {chosen[0]} + {best_cal_name}")
print("Final metrics:", best_cal_metrics)

results = pd.DataFrame([
    {"Model": "Logistic L2", **m_l2},
    {"Model": "ElasticNet", **m_en},
    {"Model": f"FINAL {chosen[0]} + {best_cal_name}", **best_cal_metrics},
]).sort_values("Brier").reset_index(drop=True)

print("\n=== Results ===")
print(results.round(5).to_string(index=False))

joblib.dump(best_cal_model, "model_LOGISTIC_PHASE1_FINAL.joblib")

MODEL_META_PATH = "model_LOGISTIC_PHASE1_FINAL_meta.joblib"
meta = {
    "feature_cols": feature_cols,          # original names (before underscore rename)
    "use_team_dummies": USE_TEAM_DUMMIES,
    "target_col": TARGET_COL,
}
joblib.dump(meta, MODEL_META_PATH)
print("✅ Saved model metadata:", MODEL_META_PATH)

results.to_csv("model_results_summary_logistic_only.csv", index=False)
print("\n✅ Saved final model + summary.")

Fitting 6 folds for each of 50 candidates, totalling 300 fits
Fitting 6 folds for each of 60 candidates, totalling 360 fits

Pre-calibration best: ElasticNet {'F1': 0.5804263371794003, 'ROC_AUC': np.float64(0.6412637139374238), 'Brier': np.float64(0.23691156774842256)}

Calibration comparison:
base      Brier=0.23691  ROC=0.64126  F1=0.58043
sigmoid   Brier=0.23666  ROC=0.64325  F1=0.59169
isotonic  Brier=0.24040  ROC=0.64162  F1=0.57334

✅ FINAL: ElasticNet + sigmoid
Final metrics: {'F1': 0.5916916916916917, 'ROC_AUC': np.float64(0.6432547744819178), 'Brier': np.float64(0.23665720734446066)}

=== Results ===
                     Model      F1  ROC_AUC   Brier
FINAL ElasticNet + sigmoid 0.59169  0.64325 0.23666
                ElasticNet 0.58043  0.64126 0.23691
               Logistic L2 0.59034  0.63149 0.23888
✅ Saved model metadata: model_LOGISTIC_PHASE1_FINAL_meta.joblib

✅ Saved final model + summary.


In [13]:
DATE_OFFSET = +0
TARGET_DATE = (datetime.now() + timedelta(days=DATE_OFFSET)).date()
print(f"📅 Target slate date set to: {TARGET_DATE}")

CSV_PATH = "odds.csv"
ONLY_PLAYED_GAMES = True  # kept for compatibility (not used elsewhere yet)

def _strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", str(s)) if not unicodedata.combining(c))

def _american_to_decimal(a):
    try:
        a = int(a)
    except Exception:
        return None
    if a == 0 or abs(a) < 100 or a == -100:
        return None
    return 1.0 + (a / 100.0) if a > 0 else 1.0 + (100.0 / abs(a))

def _parse_odds(s: str):
    if s is None:
        return (None, None, None)
    s = s.strip().lower().replace(" ", "")
    if s in {"s", "skip"}:
        return (None, None, "skip")
    if s in {"q", "quit"}:
        return (None, None, "quit")

    # accept decimal
    if "." in s or "," in s:
        try:
            dec = float(s.replace(",", "."))
            return (None, dec if dec > 1.01 else None, None)
        except Exception:
            return (None, None, None)

    m = re.fullmatch(r"([+-]?)(\d{2,4})", s)
    if not m:
        return (None, None, None)

    sign, num = m.groups()
    american = -int(num) if sign == "-" else +int(num)
    dec = _american_to_decimal(american)
    return (american, dec, None) if dec is not None else (None, None, None)

def _mk_template(df_games, target_date):
    g = df_games.copy()
    g["Date"] = pd.to_datetime(g["Date"], errors="coerce")

    # For past dates -> keep only played games; for future dates -> no filter
    today = datetime.now().date()
    if target_date < today and {"Home Score", "Away Score"}.issubset(g.columns):
        g = g[g["Home Score"].notna() & g["Away Score"].notna()]

    g = g[g["Date"].dt.date == target_date]

    out = (
        g.drop_duplicates(subset=["Date", "Home Team", "Away Team"])
         .loc[:, ["Date", "Away Team", "Home Team"]]
         .assign(**{
             "Away American Odds": None, "Away Odds": None,
             "Home American Odds": None, "Home Odds": None
         })
         .loc[:, [
             "Date", "Away Team", "Away American Odds", "Away Odds",
             "Home Team", "Home American Odds", "Home Odds"
         ]]
         .reset_index(drop=True)
    )

    out["Away Team"] = out["Away Team"].map(_strip_accents)
    out["Home Team"] = out["Home Team"].map(_strip_accents)
    return out

def _slate_counts_nearby(df_games, center_date, span=3):
    g = df_games.copy()
    g["Date"] = pd.to_datetime(g["Date"], errors="coerce")

    counts = []
    for delta in range(-1, span + 1):
        d = center_date + timedelta(days=delta)
        counts.append((d, int((g["Date"].dt.date == d).sum())))
    return counts

def enter_american_odds(df_games, date_str=None):
    target = pd.to_datetime(date_str).date() if date_str else datetime.now().date()
    tmpl = _mk_template(df_games, target)

    print(f"\nTarget date: {target} | games found: {len(tmpl)}")
    print("Nearby slates (games per day):")
    for d, n in _slate_counts_nearby(df_games, target, span=3):
        mark = "  (today)" if d == target else ""
        print(f"  {d}: {n}{mark}")

    if tmpl.empty:
        print("No games for this date in df_all. Pick another date (e.g., enter_american_odds(df_all, '2025-10-12')).")
        return tmpl

    print("\nEnter AMERICAN odds (e.g., -120, +135).")
    print("Press Enter to leave a side blank, 's' to skip a game, 'q' to quit.\n")

    def _ask_side(label, team):
        while True:
            s = input(f"  ({label})  {team}  American odds: ").strip()
            if s == "":
                return (None, None, None)
            a, dec, cmd = _parse_odds(s)
            if cmd in {"quit", "skip"}:
                return (None, None, cmd)
            if dec is not None:
                return (a, dec, None)
            print("    -> invalid (try -120, +135, or 1.95)")

    for i in range(len(tmpl)):
        away = tmpl.at[i, "Away Team"]
        home = tmpl.at[i, "Home Team"]
        header = f"[{i+1}/{len(tmpl)}]  {away}  @  {home}"
        print("=" * len(header))
        print(header)
        print("=" * len(header))

        a_american, a_decimal, cmd = _ask_side("AWAY", away)
        if cmd == "quit":
            break
        if cmd == "skip":
            print("  skipped game\n")
            continue

        h_american, h_decimal, cmd2 = _ask_side("HOME", home)
        if cmd2 == "quit":
            break
        if cmd2 == "skip":
            print("  skipped game\n")
            continue

        tmpl.at[i, "Away American Odds"] = a_american
        tmpl.at[i, "Away Odds"]          = a_decimal
        tmpl.at[i, "Home American Odds"] = h_american
        tmpl.at[i, "Home Odds"]          = h_decimal
        print()

    complete = tmpl.dropna(subset=["Away Odds", "Home Odds"]).reset_index(drop=True)
    complete.to_csv(CSV_PATH, index=False)
    print(f"\n✅ Saved {len(complete)} matchup(s) with odds to {CSV_PATH}")
    return complete

# ===== RUN =====
df_odds = enter_american_odds(df_games_master, TARGET_DATE.strftime("%Y-%m-%d"))
print("\nSaved rows preview:")
print(df_odds.head(10))

📅 Target slate date set to: 2026-02-25

Target date: 2026-02-25 | games found: 8
Nearby slates (games per day):
  2026-02-24: 0
  2026-02-25: 8  (today)
  2026-02-26: 12
  2026-02-27: 4
  2026-02-28: 13

Enter AMERICAN odds (e.g., -120, +135).
Press Enter to leave a side blank, 's' to skip a game, 'q' to quit.

[1/8]  Edmonton Oilers  @  Anaheim Ducks


  (AWAY)  Edmonton Oilers  American odds:  s


  skipped game

[2/8]  Winnipeg Jets  @  Vancouver Canucks


  (AWAY)  Winnipeg Jets  American odds:  s


  skipped game

[3/8]  Vegas Golden Knights  @  Los Angeles Kings


...

  skipped game

[7/8]  Buffalo Sabres  @  New Jersey Devils


  (AWAY)  Buffalo Sabres  American odds:  s


  skipped game

[8/8]  Seattle Kraken  @  Dallas Stars


  (AWAY)  Seattle Kraken  American odds:  s


  skipped game


✅ Saved 0 matchup(s) with odds to odds.csv

Saved rows preview:
Empty DataFrame
Columns: [Date, Away Team, Away American Odds, Away Odds, Home Team, Home American Odds, Home Odds]
Index: []


In [14]:
import numpy as np
import pandas as pd

DATE_COL = "Date"

def _assert_sorted(df, date_col):
    if not df[date_col].is_monotonic_increasing:
        df.sort_values(date_col, inplace=True, kind="mergesort")

def _latest_row_team(df_hist, team, as_of_dt, date_col=DATE_COL):
    m = (
        (df_hist[date_col] < as_of_dt) &
        ((df_hist["Home Team"] == team) | (df_hist["Away Team"] == team))
    )
    if not m.any():
        raise ValueError(f"No history for team={team} before {as_of_dt.date()}")
    return df_hist.loc[m].iloc[-1]

def _latest_value_team(df_hist, team, col_home, col_away, as_of_dt, date_col=DATE_COL):
    row = _latest_row_team(df_hist, team, as_of_dt, date_col)
    return row[col_home] if row["Home Team"] == team else row[col_away]

def build_team_state_asof(df_final, as_of_dt, date_col=DATE_COL):
    df_hist = (
        df_final.copy()
        .assign(**{date_col: pd.to_datetime(df_final[date_col], errors="coerce")})
        .dropna(subset=[date_col])
        .sort_values(date_col)
        .reset_index(drop=True)
    )
    _assert_sorted(df_hist, date_col)

    teams = pd.unique(pd.concat([df_hist["Home Team"], df_hist["Away Team"]]).astype(str))

    # Precompute rolling column pairs once (cuts a lot of repetition)
    rolling_pairs = []
    for col in df_hist.columns:
        if col.startswith("Home_") and "_L" in col:
            base = col.replace("Home_", "", 1)
            acol = "Away_" + base
            if acol in df_hist.columns:
                rolling_pairs.append((base, col, acol))  # (base_name, home_col, away_col)

    out = []
    for t in map(lambda x: str(x).strip(), teams):
        last = _latest_row_team(df_hist, t, as_of_dt, date_col)
        rest_days = (as_of_dt.normalize() - pd.to_datetime(last[date_col]).normalize()).days

        row = {"Team": t, "RestDays": rest_days, "B2B": int(rest_days == 1)}

        # Team-level features (use latest value based on whether team was home/away)
        pairs = [
            ("Last 10 Wins", "Home Last 10 Wins", "Away Last 10 Wins"),
            ("Win Rate", "Home Win Rate", "Away Win Rate"),
            ("Win Streak", "Home Team Overall Win Streak Before Game", "Away Team Overall Win Streak Before Game"),
            ("Opponent Strength", "Home Opponent Strength", "Away Opponent Strength"),
            ("SRS", "Home_SRS", "Away_SRS"),
            ("Rank", "Home_Rank", "Away_Rank"),
        ]
        for out_name, hcol, acol in pairs:
            if {hcol, acol}.issubset(df_hist.columns):
                row[out_name] = _latest_value_team(df_hist, t, hcol, acol, as_of_dt, date_col)

        # Snapshot metrics
        for m in ["PP%", "PK%", "SV%", "xGF%"]:
            hcol, acol = f"Home_{m}", f"Away_{m}"
            if {hcol, acol}.issubset(df_hist.columns):
                row[m] = _latest_value_team(df_hist, t, hcol, acol, as_of_dt, date_col)

        # Rolling metrics (base stored without Home_/Away_)
        for base, hcol, acol in rolling_pairs:
            row[base] = _latest_value_team(df_hist, t, hcol, acol, as_of_dt, date_col)

        out.append(row)

    return pd.DataFrame(out).set_index("Team").sort_index()

def build_X_slate(df_odds, df_final, feature_cols, as_of_dt, date_col=DATE_COL):
    df_od = df_odds.copy()
    df_od["Home Team"] = df_od["Home Team"].astype(str).str.strip()
    df_od["Away Team"] = df_od["Away Team"].astype(str).str.strip()

    team_state = build_team_state_asof(df_final, as_of_dt, date_col)
    X_rows = []

    for _, r in df_od.iterrows():
        h, a = r["Home Team"], r["Away Team"]
        if h not in team_state.index or a not in team_state.index:
            raise ValueError(f"Missing team in history state: {h} or {a}")

        hs, as_ = team_state.loc[h], team_state.loc[a]
        feats = {}

        # Simple diffs
        if "Rest_Diff" in feature_cols:
            feats["Rest_Diff"] = float(hs["RestDays"]) - float(as_["RestDays"])
        if "B2B_Diff" in feature_cols:
            feats["B2B_Diff"] = int(hs["B2B"]) - int(as_["B2B"])
        if "Home Advantage" in feature_cols:
            feats["Home Advantage"] = 1.0

        # Diff features derived from team_state
        diff_map = {
            "Last 10 Wins": ("Last 10 Wins", "Last 10 Wins"),
            "Win Streak Impact": ("Win Streak", "Win Streak"),
            "SRS_Diff": ("SRS", "SRS"),
            "Rank Difference": ("Rank", "Rank"),
        }
        for feat_name, (hkey, akey) in diff_map.items():
            if feat_name in feature_cols:
                feats[feat_name] = float(hs.get(hkey, np.nan)) - float(as_.get(akey, np.nan))

        # Opp strength (kept as two separate features)
        if "Home Opponent Strength" in feature_cols:
            feats["Home Opponent Strength"] = float(hs.get("Opponent Strength", np.nan))
        if "Away Opponent Strength" in feature_cols:
            feats["Away Opponent Strength"] = float(as_.get("Opponent Strength", np.nan))

        # Rolling deltas
        for c in feature_cols:
            if c.startswith("Delta_") and "_L" in c:
                metric = c.replace("Delta_", "", 1)  # ex: "PP%_L5"
                feats[c] = float(hs.get(metric, np.nan)) - float(as_.get(metric, np.nan))

        row = {fc: feats.get(fc, np.nan) for fc in feature_cols}
        row["Home_Team"] = h
        row["Away_Team"] = a
        X_rows.append(row)

    X_raw = pd.DataFrame(X_rows)
    rename_map = {c: c.replace(" ", "_") for c in X_raw.columns}
    X = X_raw.rename(columns=rename_map)
    return X, rename_map

In [15]:
MODEL_PATH = "model_LOGISTIC_PHASE1_FINAL.joblib"
MODEL_META_PATH = "model_LOGISTIC_PHASE1_FINAL_meta.joblib"

BANKROLL = 200
PREDICTIONS_CSV = "predictions.csv"
DATE_OFFSET = 0
DATE_COL_IN_HISTORY = "Date"

TARGET_DATE = (datetime.now() + timedelta(days=DATE_OFFSET)).date()
as_of_dt = pd.Timestamp(TARGET_DATE)

# ---- required objects ----
for name in ("df_odds", "df_final"):
    if name not in globals():
        raise RuntimeError(f"Missing `{name}` in memory.")

for path in (MODEL_PATH, MODEL_META_PATH):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

model = joblib.load(MODEL_PATH)
meta = joblib.load(MODEL_META_PATH)

feature_cols = meta["feature_cols"]
use_team_dummies = meta["use_team_dummies"]

# ---- prep odds first (avoid rebuilding X twice) ----
df_out = df_odds.copy()
df_out["SlateDate"] = TARGET_DATE.isoformat()

for c in ("Home Odds", "Away Odds"):
    df_out[c] = pd.to_numeric(df_out[c], errors="coerce")

df_out = df_out.dropna(subset=["Home Odds", "Away Odds"]).reset_index(drop=True)
if df_out.empty:
    raise ValueError("No rows with both Home Odds and Away Odds. Nothing to predict.")

# ---- build X slate (aligned to training meta) ----
X_slate, rename_map_live = build_X_slate(
    df_odds=df_out,
    df_final=df_final,
    feature_cols=feature_cols,
    as_of_dt=as_of_dt,
    date_col=DATE_COL_IN_HISTORY,
)

# ---- validate required columns exist ----
expected = [c.replace(" ", "_") for c in feature_cols]
if use_team_dummies:
    expected += ["Home_Team", "Away_Team"]

missing = [c for c in expected if c not in X_slate.columns]
if missing:
    raise ValueError(f"Missing columns required for inference: {missing}")

# ---- predict ----
p_home = np.clip(model.predict_proba(X_slate)[:, 1], 1e-6, 1 - 1e-6)
df_out["p_home"] = np.round(p_home, 4)

# ---- Kelly sizing ----
def _kelly(p_win: float, dec_odds: float) -> float:
    b = dec_odds - 1.0
    if b <= 0:
        return 0.0
    q = 1.0 - p_win
    return max(0.0, (b * p_win - q) / b)

home_pre = np.array([_kelly(p, o) for p, o in zip(p_home, df_out["Home Odds"])], dtype=float) * BANKROLL
away_pre = np.array([_kelly(1.0 - p, o) for p, o in zip(p_home, df_out["Away Odds"])], dtype=float) * BANKROLL

total_pre = float(np.nansum(home_pre) + np.nansum(away_pre))
scaled = BANKROLL > 0 and total_pre > BANKROLL

if scaled:
    s = BANKROLL / total_pre
    home_bet = np.round(home_pre * s).astype(int)
    away_bet = np.round(away_pre * s).astype(int)
else:
    home_bet = np.round(home_pre).astype(int)
    away_bet = np.round(away_pre).astype(int)

df_out["Scaled"] = scaled
df_out["Home Bet"] = np.where(home_bet > 0, home_bet, 0)
df_out["Away Bet"] = np.where(away_bet > 0, away_bet, 0)

cols = ["SlateDate", "Away Team", "Away Bet", "Away Odds", "Home Team", "Home Bet", "Home Odds", "p_home", "Scaled"]
print(df_out[cols])

# ---- save append ----
write_header = not os.path.exists(PREDICTIONS_CSV)
mode = "w" if write_header else "a"
df_out[cols].to_csv(PREDICTIONS_CSV, index=False, mode=mode, header=write_header)

print(f"\nSaved {len(df_out)} rows to {PREDICTIONS_CSV} (mode='{mode}', header={write_header})")

ValueError: No rows with both Home Odds and Away Odds. Nothing to predict.

In [ ]:
 df_final.to_csv("df_final.csv", index=False)
